# PPO Report

This project is to solve CartPole problem from Gym OpenAI. Following is sample code of PPO algorithm provided by https://morvanzhou.github.io/tutorials. My understanding of each line of sample code is in the inline comments, and feel free to check the source file.

To summarize, PPO algorithm is a simplified version of TRPO algorithm. It shares part of the objective function of TRPO, which is J(θ)=E\[r(θ)A^θold(s,a)\]. However, to stablize the learning process, PPO imposes a constraint by forcing r(θ), which is the ratio between new and old policy, to be within a predefined range. So the objective function of PPO is as following: J(θ)=E\[min(r(θ)A^θold(s,a), clip(r(θ),1−ϵ,1+ϵ)A^θold(s,a))\]. 

Admittedly, there are other versions of PPO. The above clipping approach was proposed by OpenAI, which DeepMind proposes another method of KL divergence. The following sample code favors the former approach. 

Following figure is the result reward of the training, where 0 is the max reward each episode can obtain. The fluctuation is still huge even with desperately small learning rate. As PPO is still actor-critic, I suspect the same problem of the update process of critic, that it uses same network to estimate current state value and next state value. If you have any idea of how to deal with the inconvergence, please let me know.

<img src="./reward.png" />

Following is sample code with my inline comments.

Part 1: hyper parameters

In [ ]:
# max episodes
EP_MAX = 1000
# max iteration in each episode
EP_LEN = 200
# discount factor
GAMMA = 0.9
# learning rate of actor and critic
A_LR = 0.00005
C_LR = 0.0001
# batch size
BATCH = 32
# update steps of actor and critic
A_UPDATE_STEPS = 10
C_UPDATE_STEPS = 10
# dimension of states and action
S_DIM, A_DIM = 3, 1
# two reward functions proposed by DeepMind and OpenAI, respectively.
# This code uses the clipping form by OpenAI
METHOD = [
    dict(name='kl_pen', kl_target=0.01, lam=0.5),   # KL penalty
    dict(name='clip', epsilon=0.2),                 # Clipped surrogate objective, find this is better
][1]        # choose the method for optimization

Part 2: construction of PPO

In [ ]:
class PPO(object):
    def __init__(self):
        self.sess = tf.Session()
        # placehold for states
        self.tfs = tf.placeholder(tf.float32, [None, S_DIM], 'state')

        # critic network
        with tf.variable_scope('critic'):
            # construct two fully connected layers for critic
            l1 = tf.layers.dense(self.tfs, 100, tf.nn.relu)
            self.v = tf.layers.dense(l1, 1)
            # placeholder for discounted reward
            self.tfdc_r = tf.placeholder(tf.float32, [None, 1], 'discounted_r')
            # Advantage = discounted_reward - estimated_value
            self.advantage = self.tfdc_r - self.v
            # loss of critic is the square of advantage
            self.closs = tf.reduce_mean(tf.square(self.advantage))
            # minimize the loss of critic 
            self.ctrain_op = tf.train.AdamOptimizer(C_LR).minimize(self.closs)

        # actor network
        # pi and oldpi are normal distributions
        # pi_params and oldpi_params are their parameters
        pi, pi_params = self._build_anet('pi', trainable=True)
        oldpi, oldpi_params = self._build_anet('oldpi', trainable=False)
        # sample from the normal distribution and get actions later
        with tf.variable_scope('sample_action'):
            self.sample_op = tf.squeeze(pi.sample(1), axis=0)       # choosing action
        with tf.variable_scope('update_oldpi'):
            self.update_oldpi_op = [oldp.assign(p) for p, oldp in zip(pi_params, oldpi_params)]

        # placeholder for action
        self.tfa = tf.placeholder(tf.float32, [None, A_DIM], 'action')
        # placeholder for advantage
        self.tfadv = tf.placeholder(tf.float32, [None, 1], 'advantage')
        # construct loss function of actor
        with tf.variable_scope('loss'):
            with tf.variable_scope('surrogate'):
                # ratio = tf.exp(pi.log_prob(self.tfa) - oldpi.log_prob(self.tfa))
                # compute ratio of new policy and old policy
                ratio = pi.prob(self.tfa) / oldpi.prob(self.tfa)
                # compute the objective function of TRPO
                surr = ratio * self.tfadv
            if METHOD['name'] == 'kl_pen':
                self.tflam = tf.placeholder(tf.float32, None, 'lambda')
                kl = tf.distributions.kl_divergence(oldpi, pi)
                self.kl_mean = tf.reduce_mean(kl)
                self.aloss = -(tf.reduce_mean(surr - self.tflam * kl))
            else:   # clipping method, find this is better
                # choose the minimal between original TPRO and clipped TPRO
                self.aloss = -tf.reduce_mean(tf.minimum(
                    surr,
                    tf.clip_by_value(ratio, 1.-METHOD['epsilon'], 1.+METHOD['epsilon'])*self.tfadv))
        # train the actor network
        with tf.variable_scope('atrain'):
            self.atrain_op = tf.train.AdamOptimizer(A_LR).minimize(self.aloss)

        tf.summary.FileWriter("log/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())

    # function to update both actor and critic networks
    def update(self, s, a, r):
        self.sess.run(self.update_oldpi_op)
        # calculate advantage
        adv = self.sess.run(self.advantage, {self.tfs: s, self.tfdc_r: r})
        # adv = (adv - adv.mean())/(adv.std()+1e-6)     # sometimes helpful

        # update actor
        if METHOD['name'] == 'kl_pen':
            for _ in range(A_UPDATE_STEPS):
                _, kl = self.sess.run(
                    [self.atrain_op, self.kl_mean],
                    {self.tfs: s, self.tfa: a, self.tfadv: adv, self.tflam: METHOD['lam']})
                if kl > 4*METHOD['kl_target']:  # this in in google's paper
                    break
            if kl < METHOD['kl_target'] / 1.5:  # adaptive lambda, this is in OpenAI's paper
                METHOD['lam'] /= 2
            elif kl > METHOD['kl_target'] * 1.5:
                METHOD['lam'] *= 2
            METHOD['lam'] = np.clip(METHOD['lam'], 1e-4, 10)    # sometimes explode, this clipping is my solution
        else:   # clipping method, find this is better (OpenAI's paper)
            [self.sess.run(self.atrain_op, {self.tfs: s, self.tfa: a, self.tfadv: adv}) for _ in range(A_UPDATE_STEPS)]

        # update critic
        [self.sess.run(self.ctrain_op, {self.tfs: s, self.tfdc_r: r}) for _ in range(C_UPDATE_STEPS)]

    # function to build actor network, returns a normal distribution and its parameters
    def _build_anet(self, name, trainable):
        with tf.variable_scope(name):
            l1 = tf.layers.dense(self.tfs, 100, tf.nn.relu, trainable=trainable)
            mu = 2 * tf.layers.dense(l1, A_DIM, tf.nn.tanh, trainable=trainable)
            sigma = tf.layers.dense(l1, A_DIM, tf.nn.softplus, trainable=trainable)
            norm_dist = tf.distributions.Normal(loc=mu, scale=sigma)
        params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=name)
        return norm_dist, params

    # function to choose action 
    def choose_action(self, s):
        s = s[np.newaxis, :]
        a = self.sess.run(self.sample_op, {self.tfs: s})[0]
        return np.clip(a, -2, 2)

    # function to estimate value
    def get_v(self, s):
        if s.ndim < 2: s = s[np.newaxis, :]
        return self.sess.run(self.v, {self.tfs: s})[0, 0]

Part 3: training

In [ ]:
env = gym.make('Pendulum-v0').unwrapped
ppo = PPO()
all_ep_r = []

for ep in range(EP_MAX):
    # get initial state
    s = env.reset()
    # create buffer for state, action and reward
    buffer_s, buffer_a, buffer_r = [], [], []
    # sum of reward each episode
    ep_r = 0
    for t in range(EP_LEN):    # in one episode
        env.render()
        # choose action based on state
        a = ppo.choose_action(s)
        # get next state, reward and end information
        s_, r, done, _ = env.step(a)
        # append them to buffers
        buffer_s.append(s)
        buffer_a.append(a)
        buffer_r.append((r+8)/8)    # normalize reward, find to be useful
        # current state = next state
        s = s_
        # increase sum of reward
        ep_r += r

        # update ppo
        if (t+1) % BATCH == 0 or t == EP_LEN-1:
            v_s_ = ppo.get_v(s_)
            discounted_r = []
            for r in buffer_r[::-1]:
                v_s_ = r + GAMMA * v_s_
                discounted_r.append(v_s_)
            discounted_r.reverse()

            bs, ba, br = np.vstack(buffer_s), np.vstack(buffer_a), np.array(discounted_r)[:, np.newaxis]
            buffer_s, buffer_a, buffer_r = [], [], []
            ppo.update(bs, ba, br)
    if ep == 0: all_ep_r.append(ep_r)
    else: all_ep_r.append(all_ep_r[-1]*0.9 + ep_r*0.1)
    print(
        'Ep: %i' % ep,
        "|Ep_r: %i" % ep_r,
        ("|Lam: %.4f" % METHOD['lam']) if METHOD['name'] == 'kl_pen' else '',
    )